## Image Sorter using Zero Shot Classification


In [1]:
# Constants

SUCCESS_PATH = '/content/'+ 'Sortedd'
FAILURE_PATH = '/content/' + 'Failed'
CLASSES = ["Pets", "Family", "School", "Events", "Else"]

In [2]:
# Imports
import os
import shutil
from PIL import Image
from PIL.ExifTags import TAGS
from transformers import pipeline
# More models in the model hub.
model_name = "openai/clip-vit-large-patch14-336"
classifier = pipeline("zero-shot-image-classification", model = model_name, use_fast=True, device = 0) # Device = 0 means use the GPU, but defaults to cpu if gpu isn't available


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [3]:
# Image folder sorter function

def categorise(dir_path):
    images_info = {}
    for entry in os.scandir(dir_path):
        if entry.is_file() and entry.name.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = entry.path
            scores = classifier(image_path, candidate_labels = CLASSES)
            predicted_class = scores[0]['label']

            # Create directories
            new_dir = os.path.join(SUCCESS_PATH, predicted_class)
            if not os.path.exists(new_dir):
                os.makedirs(new_dir)
            # Set destination path for the image
            new_path = os.path.join(new_dir, entry.name)
            images_info[image_path] = new_path

            # Copy the file instead of moving it
            if image_path in images_info:
                shutil.copy2(image_path, images_info[image_path])
            else:
                print(f"Error: No destination path determined for {image_path}. Skipping copy.")

In [4]:
categorise("/content/Test Images")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
image_to_classify = "path_to_cat_and_dog_image.jpeg"
labels_for_classification =  ["cat and dog",
                              "lion and cheetah",
                              "rabbit and lion"]
scores = classifier(image_to_classify,
                    candidate_labels = labels_for_classification)

print(f"The highest score is {scores[0]['score']:.3f} for the label {scores[0]['label']}")
